In [2]:
!pip install opencv-python

     ---------------------------------------- 38.6/38.6 MB 7.4 MB/s eta 0:00:00


In [10]:
# 필요한 라이브러리 임포트
import os
import json
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split



# 데이터 전처리 함수
def preprocess_data(json_files, img_directory):  # 이미지 디렉토리 경로를 인자로 추가
    # 이미지 데이터와 레이블을 저장할 리스트 선언
    images = []
    labels = []
    # 모든 JSON 파일에 대해 처리
    for json_file in json_files:
        # JSON 파일 불러오기
        with open(json_file, 'r') as f:
            data = json.load(f)
        
        # 이미지 경로 생성
        img_file = data['imagePath']  # JSON 파일에 기록된 이미지 파일명
        img_path = os.path.join(img_directory, img_file)  # 이미지 파일의 절대 경로 생성
        
        # 이미지 불러오기
        img = cv2.imread(img_path)

        # JSON 파일의 폴리곤 좌표를 이용해 이미지에 다각형 그리기
        for shape in data['shapes']:
            if shape['label'] == 'tom_flower_poly':
                points = np.array(shape['points'], np.int32)
                cv2.fillPoly(img, [points], (255, 255, 255))

        # 이미지 데이터와 레이블 저장
        images.append(img)
        labels.append(1 if len([shape for shape in data['shapes'] if shape['label'] == 'tom_flower_poly']) >= 3 else 0)
    
    # 리스트를 numpy array로 변환
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

# 딥러닝 모델 생성 함수
def create_model():
    # Sequential 모델 생성
    model = Sequential()
    # 첫 번째 Convolutional layer 추가. 입력 이미지의 크기는 1920x1080, 채널은 3(RGB)
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(1920, 1080, 3)))
    # MaxPooling layer 추가
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # 두 번째 Convolutional layer 추가
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    # MaxPooling layer 추가
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Flatten layer 추가. 이것을 통해 이전 Layer의 2차원 데이터를 1차원으로 만들어줍니다.
    model.add(Flatten())
    # 첫 번째 Fully connected layer(Dense) 추가
    model.add(Dense(128, activation='relu'))
    # 출력 레이어 추가. 이진 분류 문제이므로 노드 수는 1, 활성화 함수는 sigmoid
    model.add(Dense(1, activation='sigmoid'))
    # 모델 컴파일. Optimizer는 Adam, Loss function은 Binary Crossentropy 사용
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model



In [28]:
# 필요한 라이브러리 임포트
import os
import json
import cv2
import numpy as np
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# 데이터 전처리 함수
def preprocess_data(json_files, img_directory):  
    images = []
    labels = []
    for json_file in json_files:
        with open(json_file, 'r') as f:
            data = json.load(f)
        
        img_file = data['imagePath']
        img_path = os.path.join(img_directory, img_file)
        
        img = Image.open(img_path)
        img = np.array(img)
        
        if img is None:
            print(f"이미지 로드 실패: {img_path}")
            continue  

        for shape in data['shapes']:
            if shape['label'] == 'tom_flower_poly':
                points = np.array(shape['points'], np.int32)
                cv2.fillPoly(img, [points], (255, 255, 255))

        images.append(img)
        labels.append(1 if len([shape for shape in data['shapes'] if shape['label'] == 'tom_flower_poly']) >= 3 else 0)
    
    print(f"로드된 이미지 수: {len(images)}")
    images = np.array(images)
    labels = np.array(labels)
    return images, labels



In [29]:
# 딥러닝 모델 생성 함수
def create_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(1920, 1080, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

directory = 'E:/tomato_label/097.지능형_스마트팜_통합_데이터(토마토)/01.데이터/1.Training/라벨링데이터/f.만개꽃/'
json_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.json')]
json_files = json_files[:200]

img_directory = u'E:/tomato_label/097.지능형_스마트팜_통합_데이터(토마토)/01.데이터/1.Training/원천데이터/f.만개꽃/'

images, labels = preprocess_data(json_files, img_directory)
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

model = create_model()
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))


MemoryError: Unable to allocate 5.93 MiB for an array with shape (1080, 1920, 3) and data type uint8

In [30]:
# 필요한 라이브러리 임포트
import os
import json
import cv2
import numpy as np
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# 데이터 전처리 함수
def preprocess_data(json_file, img_directory):  
    with open(json_file, 'r') as f:
        data = json.load(f)

    img_file = data['imagePath']
    img_path = os.path.join(img_directory, img_file)
        
    img = Image.open(img_path)
    img = np.array(img)
        
    if img is None:
        print(f"이미지 로드 실패: {img_path}")
        return None, None

    for shape in data['shapes']:
        if shape['label'] == 'tom_flower_poly':
            points = np.array(shape['points'], np.int32)
            cv2.fillPoly(img, [points], (255, 255, 255))

    label = 1 if len([shape for shape in data['shapes'] if shape['label'] == 'tom_flower_poly']) >= 3 else 0
    return img, label

# 데이터 로드 함수
def data_generator(json_files, img_directory, batch_size):
    images = []
    labels = []
    for json_file in json_files:
        img, label = preprocess_data(json_file, img_directory)
        if img is not None:
            images.append(img)
            labels.append(label)
            if len(images) == batch_size:
                yield np.array(images), np.array(labels)
                images = []
                labels = []

# 딥러닝 모델 생성 함수
def create_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(1920, 1080, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model



In [36]:
directory = 'E:/tomato_label/097.지능형_스마트팜_통합_데이터(토마토)/01.데이터/1.Training/라벨링데이터/f.만개꽃/'
json_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.json')]
json_files = json_files[:200]

img_directory = u'E:/tomato_label/097.지능형_스마트팜_통합_데이터(토마토)/01.데이터/1.Training/원천데이터/f.만개꽃/'

train_json_files, test_json_files = train_test_split(json_files, test_size=0.2, random_state=42)

model = create_model()


BATCH_SIZE = 4  # 원하는 배치 크기를 지정

# 훈련 데이터셋에 대해 모델 학습
for imgs, labels in data_generator(train_json_files, img_directory, BATCH_SIZE):
    model.train_on_batch(imgs, labels)

# 테스트 데이터셋에 대해 모델 평가
eval_loss, eval_acc = 0, 0
n_test_samples = 0
for imgs, labels in data_generator(test_json_files, img_directory, BATCH_SIZE):
    loss, acc = model.evaluate(imgs, labels, verbose=0)
    eval_loss += loss * len(labels)
    eval_acc += acc * len(labels)
    n_test_samples += len(labels)

print(f"테스트 손실: {eval_loss / n_test_samples}")
print(f"테스트 정확도: {eval_acc / n_test_samples}")

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[8198656,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:StatelessRandomUniformV2] name: 

In [34]:
directory = 'E:/tomato_label/097.지능형_스마트팜_통합_데이터(토마토)/01.데이터/1.Training/라벨링데이터/f.만개꽃/'
json_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.json')]
json_files = json_files[:200]

img_directory = u'E:/tomato_label/097.지능형_스마트팜_통합_데이터(토마토)/01.데이터/1.Training/원천데이터/f.만개꽃/'

train_json_files, test_json_files = train_test_split(json_files, test_size=0.2, random_state=42)

model = create_model()

# 훈련 데이터셋에 대해 모델 학습
for img, label in data_generator(train_json_files, img_directory):
    img = img.reshape(-1, 1920, 1080, 3)  # 이미지 차원 변경
    #img = np.expand_dims(img, axis=0)
    label = np.array([label])
    model.train_on_batch(img, label)

# 테스트 데이터셋에 대해 모델 평가
eval_loss, eval_acc = 0, 0
n_test_samples = 0
for img, label in data_generator(test_json_files, img_directory):
    img = np.expand_dims(img, axis=0)
    label = np.array([label])
    loss, acc = model.evaluate(img, label, verbose=0)
    eval_loss += loss
    eval_acc += acc
    n_test_samples += 1

print(f"테스트 손실: {eval_loss / n_test_samples}")
print(f"테스트 정확도: {eval_acc / n_test_samples}")

TypeError: data_generator() takes 2 positional arguments but 3 were given